# بناء نموذج تصنيف: المأكولات الآسيوية والهندية اللذيذة


## مصنفات المأكولات 2

في درس التصنيف الثاني هذا، سنستكشف `طرق إضافية` لتصنيف البيانات الفئوية. كما سنتعلم عن العواقب الناتجة عن اختيار مصنف معين بدلاً من الآخر.

### [**اختبار ما قبل المحاضرة**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **المتطلبات الأساسية**

نفترض أنك قد أكملت الدروس السابقة لأننا سنواصل بعض المفاهيم التي تعلمناها سابقًا.

بالنسبة لهذا الدرس، سنحتاج إلى الحزم التالية:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) هو [مجموعة من حزم R](https://www.tidyverse.org/packages) مصممة لجعل علم البيانات أسرع وأسهل وأكثر متعة!

-   `tidymodels`: إطار عمل [tidymodels](https://www.tidymodels.org/) هو [مجموعة من الحزم](https://www.tidymodels.org/packages/) للنمذجة وتعلم الآلة.

-   `themis`: حزمة [themis](https://themis.tidymodels.org/) توفر خطوات إضافية للوصفات للتعامل مع البيانات غير المتوازنة.

يمكنك تثبيتها باستخدام:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

بدلاً من ذلك، يقوم السكربت أدناه بالتحقق مما إذا كانت الحزم المطلوبة لإكمال هذا الوحدة موجودة ويقوم بتثبيتها إذا كانت مفقودة.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. خريطة التصنيف**

في [الدرس السابق](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1)، حاولنا الإجابة على السؤال: كيف نختار بين نماذج متعددة؟ إلى حد كبير، يعتمد ذلك على خصائص البيانات ونوع المشكلة التي نريد حلها (على سبيل المثال التصنيف أو الانحدار؟)

سابقًا، تعلمنا عن الخيارات المختلفة المتاحة لتصنيف البيانات باستخدام ورقة الغش الخاصة بمايكروسوفت. إطار عمل التعلم الآلي في Python، Scikit-learn، يقدم ورقة غش مشابهة ولكن أكثر تفصيلًا يمكن أن تساعدك بشكل أكبر في تضييق نطاق اختيارك للمقدّرين (مصطلح آخر للمصنفات):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> نصيحة: [قم بزيارة هذه الخريطة عبر الإنترنت](https://scikit-learn.org/stable/tutorial/machine_learning_map/) وانقر على المسار لقراءة الوثائق.
>
> يوفر [موقع مرجع Tidymodels](https://www.tidymodels.org/find/parsnip/#models) أيضًا وثائق ممتازة حول الأنواع المختلفة من النماذج.

### **الخطة** 🗺️

هذه الخريطة مفيدة جدًا بمجرد أن تكون لديك فهم واضح لبياناتك، حيث يمكنك "السير" على طول مساراتها لاتخاذ قرار:

- لدينا أكثر من 50 عينة

- نريد التنبؤ بفئة معينة

- لدينا بيانات معنونة

- لدينا أقل من 100 ألف عينة

- ✨ يمكننا اختيار Linear SVC

- إذا لم ينجح ذلك، وبما أن لدينا بيانات رقمية

    - يمكننا تجربة ✨ KNeighbors Classifier

        - إذا لم ينجح ذلك، جرب ✨ SVC و ✨ Ensemble Classifiers

هذا مسار مفيد جدًا للاتباع. الآن، دعونا نبدأ باستخدام إطار عمل النمذجة [tidymodels](https://www.tidymodels.org/): مجموعة متناسقة ومرنة من حزم R تم تطويرها لتشجيع الممارسات الإحصائية الجيدة 😊.

## 2. تقسيم البيانات والتعامل مع مجموعة بيانات غير متوازنة.

من دروسنا السابقة، تعلمنا أن هناك مجموعة من المكونات المشتركة عبر مطابخنا. أيضًا، كان هناك توزيع غير متساوٍ إلى حد كبير في عدد المطابخ.

سنتعامل مع هذه الأمور من خلال:

- حذف المكونات الأكثر شيوعًا التي تسبب ارتباكًا بين المطابخ المختلفة، باستخدام `dplyr::select()`.

- استخدام `recipe` لمعالجة البيانات وتجهيزها للنمذجة من خلال تطبيق خوارزمية `over-sampling`.

لقد تناولنا ما سبق في الدرس السابق، لذا يجب أن يكون هذا سهلًا جدًا 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### التعامل مع البيانات غير المتوازنة

البيانات غير المتوازنة غالبًا ما تؤثر سلبًا على أداء النموذج. العديد من النماذج تعمل بشكل أفضل عندما يكون عدد الملاحظات متساويًا، وبالتالي تواجه صعوبة مع البيانات غير المتوازنة.

هناك طريقتان رئيسيتان للتعامل مع مجموعات البيانات غير المتوازنة:

- إضافة ملاحظات إلى الفئة الأقل: `الإفراط في أخذ العينات` مثل استخدام خوارزمية SMOTE التي تقوم بتوليد أمثلة جديدة للفئة الأقل بشكل اصطناعي باستخدام أقرب الجيران لهذه الحالات.

- إزالة ملاحظات من الفئة الأكثر: `التقليل من أخذ العينات`

في درسنا السابق، قمنا بتوضيح كيفية التعامل مع مجموعات البيانات غير المتوازنة باستخدام `وصفة`. يمكن اعتبار الوصفة بمثابة مخطط يصف الخطوات التي يجب تطبيقها على مجموعة البيانات لجعلها جاهزة لتحليل البيانات. في حالتنا، نريد أن يكون لدينا توزيع متساوٍ في عدد أنواع المأكولات في مجموعة `التدريب`. لنبدأ مباشرة.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

الآن نحن جاهزون لتدريب النماذج 👩‍💻👨‍💻!

## 3. ما بعد نماذج الانحدار متعدد الحدود

في درسنا السابق، تناولنا نماذج الانحدار متعدد الحدود. دعونا نستكشف بعض النماذج الأكثر مرونة للتصنيف.

### آلات الدعم المتجهة

في سياق التصنيف، تعد `آلات الدعم المتجهة` تقنية تعلم آلي تهدف إلى إيجاد *مستوى فائق* يفصل بين الفئات "بأفضل" طريقة ممكنة. دعونا نلقي نظرة على مثال بسيط:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ لا تفصل بين الفئات. H2~ تفصل بينها ولكن بهامش صغير فقط. H3~ تفصل بينها بأكبر هامش ممكن.

#### المصنف الخطي لدعم المتجهات

التجميع باستخدام دعم المتجهات (SVC) هو أحد أفراد عائلة تقنيات تعلم الآلة الخاصة بدعم المتجهات. في SVC، يتم اختيار المستوى الفاصل بحيث يفصل بشكل صحيح بين `معظم` ملاحظات التدريب، ولكنه `قد يخطئ` في تصنيف بعض الملاحظات. من خلال السماح لبعض النقاط بأن تكون على الجانب الخطأ، يصبح SVM أكثر قوة في مواجهة القيم الشاذة وبالتالي يقدم تعميمًا أفضل للبيانات الجديدة. يُطلق على المعامل الذي ينظم هذا الانتهاك اسم `التكلفة`، والذي تكون قيمته الافتراضية 1 (راجع `help("svm_poly")`).

لنقم بإنشاء SVC خطي عن طريق ضبط `degree = 1` في نموذج SVM متعدد الحدود.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

الآن بعد أن قمنا بتجميع خطوات المعالجة المسبقة وتحديد النموذج في *سير العمل*، يمكننا المضي قدمًا وتدريب SVC الخطي وتقييم النتائج أثناء ذلك. بالنسبة لمقاييس الأداء، دعونا ننشئ مجموعة مقاييس لتقييم: `accuracy`، `sensitivity`، `Positive Predicted Value` و `F Measure`.

> `augment()` سيضيف عمودًا (أعمدة) للتنبؤات إلى البيانات المقدمة.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### آلة المتجهات الداعمة

آلة المتجهات الداعمة (SVM) هي امتداد لمصنف المتجهات الداعمة لتتمكن من التعامل مع الحدود غير الخطية بين الفئات. في جوهرها، تستخدم SVMs *حيلة النواة* لتوسيع مساحة الميزات لتتكيف مع العلاقات غير الخطية بين الفئات. إحدى وظائف النواة الشهيرة والمرنة للغاية التي تستخدمها SVMs هي *وظيفة الأساس الشعاعي.* دعونا نرى كيف ستؤدي على بياناتنا.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

أفضل بكثير 🤩!

> ✅ يرجى الاطلاع على:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html)، التعلم الآلي العملي باستخدام R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/)، مقدمة في التعلم الإحصائي مع تطبيقات باستخدام R
>
> لمزيد من القراءة.

### مصنفات أقرب الجيران

*K*-أقرب الجيران (KNN) هو خوارزمية يتم فيها التنبؤ بكل ملاحظة بناءً على *تشابهها* مع الملاحظات الأخرى.

دعونا نقوم بتطبيقها على بياناتنا.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

يبدو أن هذا النموذج لا يعمل بشكل جيد. ربما تغيير معايير النموذج (راجع `help("nearest_neighbor")`) سيحسن أداء النموذج. تأكد من تجربته.

> ✅ يرجى الاطلاع على:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> للتعرف أكثر على مصنفات *K*-Nearest Neighbors.

### المصنفات التجميعية

تعمل خوارزميات التجميع عن طريق دمج عدة نماذج أساسية لإنتاج نموذج مثالي إما عن طريق:

`bagging`: تطبيق *دالة متوسط* على مجموعة من النماذج الأساسية.

`boosting`: بناء سلسلة من النماذج التي تعتمد على بعضها البعض لتحسين الأداء التنبؤي.

لنبدأ بتجربة نموذج الغابة العشوائية (Random Forest)، الذي يقوم ببناء مجموعة كبيرة من أشجار القرار ثم يطبق دالة متوسط للحصول على نموذج أفضل بشكل عام.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

عمل رائع 👏!

دعونا نجرب أيضًا نموذج الشجرة المعززة.

تعرف الشجرة المعززة بأنها طريقة تجميعية تقوم بإنشاء سلسلة من أشجار القرار المتتابعة، حيث تعتمد كل شجرة على نتائج الأشجار السابقة في محاولة لتقليل الخطأ تدريجيًا. تركز على أوزان العناصر التي تم تصنيفها بشكل غير صحيح وتقوم بتعديل التناسب للمصنف التالي لتصحيحها.

هناك طرق مختلفة لتطبيق هذا النموذج (راجع `help("boost_tree")`). في هذا المثال، سنقوم بتطبيق الأشجار المعززة باستخدام محرك `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ يرجى الاطلاع على:
>
> -   [التعلم الآلي للعلماء الاجتماعيين](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [التعلم الآلي العملي باستخدام R](https://bradleyboehmke.github.io/HOML/)
>
> -   [مقدمة في التعلم الإحصائي مع تطبيقات في R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - يستعرض نموذج AdaBoost الذي يُعد بديلاً جيدًا لـ xgboost.
>
> لمعرفة المزيد عن المصنفات المجمعة.

## 4. إضافي - مقارنة نماذج متعددة

قمنا بتطبيق عدد كبير من النماذج في هذا المختبر 🙌. قد يصبح من الممل أو المرهق إنشاء العديد من سير العمل من مجموعات مختلفة من المعالجات المسبقة و/أو مواصفات النماذج، ثم حساب مقاييس الأداء واحدًا تلو الآخر.

دعونا نرى ما إذا كان بإمكاننا معالجة هذا الأمر عن طريق إنشاء دالة تقوم بتطبيق قائمة من سير العمل على مجموعة التدريب، ثم تعيد مقاييس الأداء بناءً على مجموعة الاختبار. سنستخدم `map()` و `map_dfr()` من حزمة [purrr](https://purrr.tidyverse.org/) لتطبيق الدوال على كل عنصر في القائمة.

> دوال [`map()`](https://purrr.tidyverse.org/reference/map.html) تتيح لك استبدال العديد من الحلقات التكرارية بكود أكثر اختصارًا وأسهل قراءة. أفضل مكان لتعلم المزيد عن دوال [`map()`](https://purrr.tidyverse.org/reference/map.html) هو [فصل التكرار](http://r4ds.had.co.nz/iteration.html) في كتاب R for Data Science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) تتيح للمستخدمين إنشاء وتطبيق عدد كبير من النماذج بسهولة، لكنها مصممة بشكل أساسي للعمل مع تقنيات إعادة أخذ العينات مثل `cross-validation`، وهي طريقة لم نتناولها بعد.

## **🚀التحدي**

كل واحدة من هذه التقنيات تحتوي على عدد كبير من المعاملات التي يمكنك تعديلها، مثل `cost` في SVMs، و`neighbors` في KNN، و`mtry` (المتنبئات المختارة عشوائيًا) في Random Forest.

ابحث عن القيم الافتراضية لكل واحدة وفكر في ما يعنيه تعديل هذه القيم بالنسبة لجودة النموذج.

للتعرف على المزيد حول نموذج معين ومعاملاته، استخدم: `help("model")` مثلًا `help("rand_forest")`.

> في الممارسة العملية، نقوم عادةً *بتقدير* *أفضل القيم* لهذه المعاملات عن طريق تدريب العديد من النماذج على `مجموعة بيانات محاكاة` وقياس مدى أداء هذه النماذج. تُعرف هذه العملية باسم **الضبط**.

### [**اختبار ما بعد المحاضرة**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **المراجعة والدراسة الذاتية**

هناك الكثير من المصطلحات في هذه الدروس، لذا خذ دقيقة لمراجعة [هذه القائمة](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) من المصطلحات المفيدة!

#### شكر خاص إلى:

[`Allison Horst`](https://twitter.com/allison_horst/) على إنشاء الرسوم التوضيحية الرائعة التي تجعل R أكثر ترحيبًا وجاذبية. يمكنك العثور على المزيد من الرسوم التوضيحية في [معرضها](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) و[Jen Looper](https://www.twitter.com/jenlooper) على إنشاء النسخة الأصلية من هذا المقرر بلغة Python ♥️

تعلم سعيد،

[Eric](https://twitter.com/ericntay)، سفير طلاب Microsoft Learn الذهبي.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>عمل فني بواسطة @allison_horst</figcaption>



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
